<a href="https://colab.research.google.com/github/KavyaSeelam2004/caloriesidp/blob/main/CALORIES_PREDICTION_IDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pickle

import warnings
from warnings import filterwarnings
filterwarnings("ignore")

sns.set()

In [ ]:
#Load the Calories dataset
import pandas as pd

df1 = pd.read_csv("/content/calories.csv")
df1.head()

In [ ]:
df1.shape

In [ ]:
df2 = pd.read_csv("/content/exercise.csv")
df2.head()

In [ ]:
df2.shape

In [ ]:
df = pd.concat([df2,df1["Calories"]],axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(columns = ["User_ID"],axis=1,inplace =True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
cat_col=[col for col in df.columns if df[col].dtype=='O'] #-->Object-"o"
cat_col

In [ ]:
df["Gender"].value_counts()

In [ ]:
pd.get_dummies(df["Gender"],drop_first=True)

In [ ]:
categorical = df[cat_col]
categorical.head()


In [ ]:
categorical = pd.get_dummies(categorical["Gender"],drop_first=True)

In [ ]:
categorical

In [ ]:
Num_col = [col for col in df.columns if df[col].dtype != "O"]
Num_col

In [ ]:
df[Num_col].shape

In [ ]:
Numerical = df[Num_col]
Numerical.head()

In [ ]:
Numerical.shape


In [ ]:
data = pd.concat([categorical,Numerical],axis=1)

In [ ]:
data.head()


In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
# Ensure your dataframe is correctly named. If it's 'data', change '_df_4' to 'data'
data.groupby('male').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
data.columns

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# Convert boolean 'male' to int in the 'data' DataFrame
data['male'] = data['male'].astype(int)

# Define features (excluding target variable 'Calories')
features = data.drop('Calories', axis=1) # Assuming 'Calories' is in 'data' DataFrame

# Standardize the data
scaler = StandardScaler()
standardized_data = scaler.fit_transform(features)

# Apply PCA
pca = PCA(n_components=2)  # Keep 2 principal components
pca_result = pca.fit_transform(standardized_data)

# Explained variance
explained_variance = pca.explained_variance_ratio_
print("Explained Variance by Principal Components:", explained_variance)

# Create a DataFrame with PCA results
pca_df = pd.DataFrame(data=pca_result, columns=['PC1', 'PC2'])
pca_df['Calories'] = data['Calories'].values  # Include target variable

print(pca_df)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into features (X) and target variable (y)
X = df.drop('Calories', axis=1)  # Features
y = df['Calories']  # Target

# Split the dataset into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the shapes of the resulting datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


In [ ]:
# Example: One-hot encoding categorical variables in the feature matrix
X = pd.get_dummies(X, drop_first=True)  # Use drop_first=True to avoid the dummy variable trap


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np


# One-hot encoding categorical features
X = pd.get_dummies(X, drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# List of models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "XGBoost": XGBRegressor(),
    "SVR": SVR(),
    "Extra Trees": ExtraTreesRegressor(random_state=42)
}

# Dictionary to store metrics results for each model
metrics_results = {}

# Train each model and evaluate on the test set
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_test)  # Make predictions on the test set

    # Calculate regression metrics
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)

    # Store the results for each metric
    metrics_results[name] = {
        "R-squared": r2,
        "Mean Squared Error (MSE)": mse,
        "Mean Absolute Error (MAE)": mae,
        "Root Mean Squared Error (RMSE)": rmse
    }

# Create a DataFrame to display all metrics for each model
metrics_df = pd.DataFrame(metrics_results).T  # Transpose to show models as rows

# Find the best model based on R-squared
best_model_name = max(metrics_results, key=lambda x: metrics_results[x]['R-squared'])
best_model_score = metrics_results[best_model_name]['R-squared']

# Print the best model and its R-squared score
print(f"Best Model: {best_model_name}")
print(f"R-squared Accuracy: {best_model_score:.4f}")

# Print all models' regression metrics
print("\nAll Models' Regression Metrics:")
print(metrics_df.round(4))  # Rounding for better readability


In [ ]:
pip install numpy pandas matplotlib seaborn scikit-learn xgboost

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style for the plots
sns.set(style="whitegrid")

# Define the metric to visualize (you can change this to 'Mean Absolute Error', 'Mean Squared Error', etc.)
metric_to_visualize = 'R-squared'  # Change this to 'MAE', 'MSE', or 'RMSE' as needed

# Create a bar plot for the chosen metric
plt.figure(figsize=(12, 6))
sns.barplot(x=metrics_df.index, y=metrics_df[metric_to_visualize], palette='viridis')

# Add titles and labels
plt.title(f'Comparison of Regression Models: {metric_to_visualize}', fontsize=16)
plt.xlabel('Models', fontsize=14)
plt.ylabel(metric_to_visualize, fontsize=14)
plt.xticks(rotation=45)  # Rotate x labels for better readability

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming you already have the metrics_df DataFrame from previous calculations
# Here is a mock-up of how it would look. Please replace this with your actual metrics_df.
metrics_results = {
    "Linear Regression": {"R-squared": 0.85, "Mean Squared Error (MSE)": 1.2, "Mean Absolute Error (MAE)": 0.8, "Root Mean Squared Error (RMSE)": 1.1},
    "Ridge Regression": {"R-squared": 0.84, "Mean Squared Error (MSE)": 1.3, "Mean Absolute Error (MAE)": 0.9, "Root Mean Squared Error (RMSE)": 1.14},
    "Lasso Regression": {"R-squared": 0.83, "Mean Squared Error (MSE)": 1.4, "Mean Absolute Error (MAE)": 1.0, "Root Mean Squared Error (RMSE)": 1.18},
    "Decision Tree": {"R-squared": 0.78, "Mean Squared Error (MSE)": 1.6, "Mean Absolute Error (MAE)": 1.2, "Root Mean Squared Error (RMSE)": 1.24},
    "Random Forest": {"R-squared": 0.90, "Mean Squared Error (MSE)": 0.9, "Mean Absolute Error (MAE)": 0.7, "Root Mean Squared Error (RMSE)": 0.95},
    "Gradient Boosting": {"R-squared": 0.88, "Mean Squared Error (MSE)": 1.1, "Mean Absolute Error (MAE)": 0.75, "Root Mean Squared Error (RMSE)": 1.05},
    "AdaBoost": {"R-squared": 0.82, "Mean Squared Error (MSE)": 1.5, "Mean Absolute Error (MAE)": 1.1, "Root Mean Squared Error (RMSE)": 1.22},
    "XGBoost": {"R-squared": 0.91, "Mean Squared Error (MSE)": 0.85, "Mean Absolute Error (MAE)": 0.65, "Root Mean Squared Error (RMSE)": 0.92},
    "SVR": {"R-squared": 0.80, "Mean Squared Error (MSE)": 1.4, "Mean Absolute Error (MAE)": 0.85, "Root Mean Squared Error (RMSE)": 1.18},
    "Extra Trees": {"R-squared": 0.89, "Mean Squared Error (MSE)": 0.95, "Mean Absolute Error (MAE)": 0.72, "Root Mean Squared Error (RMSE)": 0.97},
}

# Create a DataFrame from the metrics results
metrics_df = pd.DataFrame(metrics_results).T  # Transpose to show models as rows

# Set the style for the plots
sns.set(style="whitegrid")

# Metrics to visualize
metrics_to_visualize = ['R-squared', 'Mean Absolute Error (MAE)', 'Mean Squared Error (MSE)', 'Root Mean Squared Error (RMSE)']

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

# Flatten the axes array for easier iteration
axes = axes.flatten()

for ax, metric in zip(axes, metrics_to_visualize):
    sns.barplot(x=metrics_df.index, y=metrics_df[metric], palette='viridis', ax=ax)
    ax.set_title(f'Comparison of Regression Models: {metric}', fontsize=14)
    ax.set_xlabel('Models', fontsize=12)
    ax.set_ylabel(metric, fontsize=12)
    ax.tick_params(axis='x', rotation=45)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import xgboost as xgb

# Sample DataFrame creation (replace this with your actual DataFrame)
data = {
    'male': [True, False, True, False, False],
    'Age': [68, 20, 69, 34, 27],
    'Height': [190.0, 166.0, 179.0, 179.0, 154.0],
    'Weight': [94.0, 60.0, 79.0, 71.0, 58.0],
    'Duration': [29.0, 14.0, 5.0, 13.0, 10.0],
    'Heart_Rate': [105.0, 94.0, 88.0, 100.0, 81.0],
    'Body_Temp': [40.8, 40.3, 38.7, 40.5, 39.8],
    'Calories': [231.0, 66.0, 26.0, 71.0, 35.0]
}

# Create DataFrame
df = pd.DataFrame(data)

# Convert 'male' column to numeric (1 for True, 0 for False)
df['male'] = df['male'].astype(int)

# Split the data into features and target variable
X = df.drop(columns=['Calories'])  # Features
y = df['Calories']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror')  # Define model with parameters
model_xgb.fit(X_train, y_train)  # Fit the model

# Make predictions from the XGBoost model
y_pred = model_xgb.predict(X_test)

# Calculate R-squared and RMSE
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Print R-squared and RMSE
print(f"R-squared: {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

# Calculate residuals
residuals = y_test - y_pred

# Plot residuals
plt.figure(figsize=(12, 6))

# Residuals Histogram
plt.subplot(1, 2, 1)
sns.histplot(residuals, bins=30, kde=True, color='blue')
plt.title('Residuals Histogram')
plt.xlabel('Residuals')
plt.ylabel('Frequency')

# Residuals vs Fitted values
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuals, alpha=0.5, color='orange')
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')

plt.tight_layout()
plt.show()

# Feature Importance
importances = model_xgb.feature_importances_  # Get feature importances
feature_names = X_train.columns  # Get feature names

# Create a DataFrame for visualization
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df.head(10), palette='viridis')
plt.title('Top 10 Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Sample data
data = {
    'male': [True, False, True, False, False],
    'Age': [68, 20, 69, 34, 27],
    'Height': [190.0, 166.0, 179.0, 179.0, 154.0],
    'Weight': [94.0, 60.0, 79.0, 71.0, 58.0],
    'Duration': [29.0, 14.0, 5.0, 13.0, 10.0],
    'Heart_Rate': [105.0, 94.0, 88.0, 100.0, 81.0],
    'Body_Temp': [40.8, 40.3, 38.7, 40.5, 39.8],
    'Calories': [231.0, 66.0, 26.0, 71.0, 35.0]
}

# Create DataFrame
df = pd.DataFrame(data)

# Encode 'male' as 1 or 0
df['male'] = df['male'].astype(int)

# Split data into features and target
X = df.drop('Calories', axis=1)
y = df['Calories']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the XGBoost model
model_xgb = xgb.XGBRegressor(objective='reg:squarederror')  # Specify objective
model_xgb.fit(X_train, y_train)

# Function to take user input and predict calories
def predict_calories(model):
    # Create a dictionary to hold user inputs
    user_input = {}

    # Collect user input for each feature with proper prompts
    user_input['Age'] = float(input("Enter Age: "))
    user_input['Height'] = float(input("Enter Height (in cm): "))
    user_input['Weight'] = float(input("Enter Weight (in kg): "))
    user_input['Duration'] = float(input("Enter Duration (in minutes): "))
    user_input['Heart_Rate'] = float(input("Enter Heart Rate: "))
    user_input['Body_Temp'] = float(input("Enter Body Temperature (in °C): "))

    # Collect input for male and validate
    while True:
        male_input = input("Are you male? (1 for Yes, 0 for No): ").strip()
        if male_input in ['1', '0']:
            user_input['male'] = int(male_input)  # Convert input to int
            break
        else:
            print("Invalid input. Please enter '1' for Yes or '0' for No.")

    # Convert user input to DataFrame
    input_df = pd.DataFrame([user_input])

    # Reorder the columns to match the training data
    input_df = input_df[X.columns]  # Ensure the order matches the training set

    # Make prediction
    prediction = model.predict(input_df)

    # Print the prediction
    print(f"Predicted Calories burned: {prediction[0]:.2f}")

# Example usage
if __name__ == "__main__":
    predict_calories(model_xgb)
